In [1]:
import os
import optuna
import pandas as pd
from time import sleep
from scipy.special import logit
from scipy.stats import beta

/home/roger/anaconda3/envs/bayes_topic_measures/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Generate and save inferred model if does not exist
if not os.path.isfile(f"bayesian_model/glmm.pickle"):
    os.system("python3 infer_bayesian_model.py")

In [3]:
def run_command_gnome(command, title="test", keep_window=False, test=False):
    if keep_window:
        command_suffix = "; $SHELL"
    else:
        command_suffix = ""
        
    if test:
        return f"""gnome-terminal --title="{title}" --command="bash -c '{command} {command_suffix}'" """
    else:
        os.system(f"""gnome-terminal --title="{title}" --command="bash -c '{command} {command_suffix}'" """)

In [4]:
def beta_ci(n_success, n_fail, level=0.95):
    a = n_success + 1
    b = n_fail+1
    l_perc = (1-level)/2
    u_perc = 1-(1-level)/2
    return beta.ppf(l_perc, a, b), beta.ppf(u_perc, a, b)

In [80]:
def generate_sim_settings(p_diff, n_raters, scores_per_r, total_scores, trials_per_sim=100):
    os.system(f"""
        python3 generate_sim_settings.py --p_diff "{p_diff}" --n_raters "{n_raters}" \
        --scores_per_r {scores_per_r} --total_scores {total_scores} --n_sims 1 \
        --trials_per_sim {trials_per_sim} --sim_name {sim_name}
    """)
    return()

In [45]:
def collate_results(sim_name, test="bht", crit_val=0.05):
    try:
        df = pd.read_csv(f"data/simulations/{sim_name}.csv")
        success = (df[f"{test}_pval"] < crit_val).sum()
        fail = (df[f"{test}_pval"] >= crit_val).sum()
        return success, fail
    except FileNotFoundError:
        return 0, 0

In [103]:
def transfer_log(sim_name, power_id, sim_id):
    df = pd.read_csv(f"data/simulations/{sim_name}.csv")
    df.insert(0, "power_id", power_id)
    df["sim_id"] = sim_id
    
    # Overwrite file if start of a new power analysis
    if sim_id == power_id == 0:
        df.to_csv(f"data/simulations/power_{sim_name}.csv", index=False)
    else:
        log = pd.read_csv(f"data/simulations/power_{sim_name}.csv")
        pd.concat([log, df]).to_csv(f"data/simulations/power_{sim_name}.csv", index=False)
        
    return

In [105]:
def objective(trial):
#         Function to optimise
        n_raters = trial.suggest_int("n_raters", 30, 200)

        generate_sim_settings(p_diff, n_raters, scores_per_r, total_scores)

        # Run simulations
        setting_files = [os.path.isfile(f"data/{sim_name}/sim_settings_{i}.csv") for i in range(N_PROCESSES)]
        while sum(setting_files)>0:
            setting_files = [os.path.isfile(f"data/{sim_name}/sim_settings_{i}.csv") for i in range(N_PROCESSES)]
            process_active = [os.path.isfile(f"data/{sim_name}/process_{i}_running") for i in range(N_PROCESSES)]

            for i in range(N_PROCESSES):
                if setting_files[i] and not process_active[i]:
                    source = "source ~/anaconda3/etc/profile.d/conda.sh\n"
                    conda = "conda activate bayes_topic_measures\n"
                    c = f"""python3 perform_sig_test.py --trials_per_sim {trials_per_sim} --process {i} --sim_name {sim_name} --n_runs {N_RUNS}"""
                    print(c)
                    run_command_gnome(source+conda+c, title=f"p{i}", keep_window=False)

            n_success, n_fail = collate_results(sim_name)
            lower_ci, upper_ci = beta_ci(n_success, n_fail)
            
            if 0.9 < lower_ci or upper_ci < 0.9:
                # Waits for all processes to complete
                while sum(process_active) > 0:
                    process_active = [os.path.isfile(f"data/{sim_name}/process_{i}_running") for i in range(N_PROCESSES)]
                    sleep(1)
                transfer_log(sim_name, power_id, sim_id[0])
                sim_id[0] += 1
                return abs(logit(0.9) - logit(n_success/(n_success+n_fail)))
            
            sleep(5)

        transfer_log(sim_name, power_id, sim_id[0])
        sim_id[0] += 1
        return abs(logit(0.9) - logit(n_success/(n_success+n_fail)))

In [94]:
# Settings for hoyle's significance testing
hoyle_total_scores = 50*26
hoyle_p_diff = 0.055
hoyle_scores_per_r = 38

In [110]:
# Simulation settings
N_PROCESSES = 2
sim_name = "test"
trials_per_sim = 100

p_diff = hoyle_p_diff
n_raters = None # Variable to be optimised by optuna 
scores_per_r = hoyle_scores_per_r
total_scores = None

# Power settings
N_RUNS = 2
# sim_id = [0] # list is used so that sim_id can be incremented in objective func  
# power_id = 0

In [111]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20)

[I 2022-10-30 13:54:12,143] A new study created in memory with name: no-name-4c742820-f125-4038-88df-0b0d35e7f0e5



    p_diff=0.055, <class 'float'>
    n_raters=39, <class 'int'>
    scores_per_r=38, <class 'int'>
    total_scores=None, <class 'NoneType'>
    n_sims=1, <class 'int'>
    trials_per_sim=100
    seed=42, <class 'int'>
    sim_name=test, <class 'str'>
    chain_method=vectorized, <class 'str'>
    clean_dir=True, <class 'bool'>
    
python3 perform_sig_test.py --trials_per_sim 100 --process 0 --sim_name test --n_runs 2


# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.


python3 perform_sig_test.py --trials_per_sim 100 --process 1 --sim_name test --n_runs 2


# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
[I 2022-10-30 13:54:54,647] Trial 0 finished with value: inf and parameters: {'n_raters': 39}. Best is trial 0 with value: inf.


...

# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.


python3 perform_sig_test.py --trials_per_sim 100 --process 0 --sim_name test --n_runs 2


# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.


python3 perform_sig_test.py --trials_per_sim 100 --process 1 --sim_name test --n_runs 2


# Option “--command” is deprecated and might be removed in a later version of gnome-terminal.
# Use “-- ” to terminate the options and put the command line to execute after it.
[I 2022-10-30 14:19:17,085] Trial 3 finished with value: inf and parameters: {'n_raters': 194}. Best is trial 1 with value: 0.9650808960435875.


In [ ]:
studies = {}
bht_n_raters = {}
for power_id, p_diff in enumerate(np.geomrange(0.02, 0.2, 20)):
    sim_id = [0]
    studies[power_id] = optuna.create_study(direction="minimize")
    studies[power_id].optimize(objective, n_trials=20)
    bht_n_raters[power_id] = studies[power_id].best_params["n_raters"]